In [1]:
%load_ext autoreload
%autoreload 2

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from darts import TimeSeries
from darts.metrics import mape, r2_score, rmse
from darts.utils import statistics
from darts.utils.data import tabularization as tab


import utils


df = utils.load_data()

/Users/Xavier/cdpq-ia/utils.py:233: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  autres = autres.apply(pd.to_numeric, errors="ignore")


# Plot raw values

In [2]:
def do_plot(col, should_diff=False, diff_order=1):
    if should_diff:
        df[col].diff(diff_order).plot(title=col)
    else:
        df[col].plot(title=col)

    plt.show()


interact(
    do_plot,
    col=widgets.Dropdown(options=df.columns, value="US_TB_YIELD_10YRS"),
    should_diff=False,
    diff_order=(1, 12, 1)
)

interactive(children=(Dropdown(description='col', index=3, options=('FFED', 'US_PERSONAL_SPENDING_PCE', 'US_CP…

<function __main__.do_plot(col, should_diff=False, diff_order=1)>

# Preprocess values

Variables that should be differenced are:
- FFED
- US_PERSONAL_SPENDING_PCE
- US_CPI
- US_TB_YIELD_*
- US_UNEMPLOYMENT_RATE (YOY)
- SNP_500 (YOY)

Variables that should not be differenced are:
- NEWS_SENTIMENT


In [3]:
# Load data
df = utils.load_data()

# Convert the dataframe to multiple TimeSeries objects (one per pre-processing method)
ts, covars_diff, covars_diff_yoy, covars_nodiff = utils.df2ts(df)

# Scale the time series and covariates
covars_diff_pipeline, covars_diff_scaled = utils.scale_ts(
    covars_diff, should_diff=True, diff_order=1
)
covars_nodiff_pipeline, covars_nodiff_scaled = utils.scale_ts(
    covars_nodiff, should_diff=False
)
covars_diff_yoy_pipeline, covars_diff_yoy_scaled = utils.scale_ts(
    covars_diff_yoy, should_diff=True, diff_order=12
)
pipeline, ts_scaled = utils.scale_ts(ts, should_diff=True, diff_order=1)


# Ensure that all timeseries are aligned (i.e. have the same time range). This is required for stacking them together.
covars_diff_scaled = covars_diff_scaled.slice_intersect(covars_diff_yoy_scaled)
covars_nodiff_scaled = covars_nodiff_scaled.slice_intersect(covars_diff_yoy_scaled)
ts_scaled = ts_scaled.slice_intersect(covars_diff_yoy_scaled)

# Stack the covariates
covariates_scaled = covars_diff_scaled.stack(covars_nodiff_scaled).stack(
    covars_diff_yoy_scaled
)


# Split the data into training and validation sets
train_ts, val_ts = ts.split_after(pd.Timestamp("2016-01-01"))
train_ts_scaled, val_ts_scaled = ts_scaled.split_after(pd.Timestamp("2016-01-01"))
train_covariates_scaled, val_covariates_scaled = covariates_scaled.split_after(
    pd.Timestamp("2016-01-01")
)

# Reconvert to DataFrame since statsmodels does not support TimeSeries objects
train = train_ts_scaled.pd_dataframe()
validation = val_ts_scaled.pd_dataframe()
history = pd.concat([train, validation])

/Users/Xavier/cdpq-ia/utils.py:233: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  autres = autres.apply(pd.to_numeric, errors="ignore")


# Plot processed values

In [ ]:
def do_plot(col):
    covariates_scaled[col].plot()
    plt.show()


interact(
    do_plot,
    col=widgets.Dropdown(options=list(covariates_scaled.components), value="US_CPI"),
)

interactive(children=(Dropdown(description='col', index=22, options=('FFED', 'US_TB_YIELD_1YR', 'US_TB_YIELD_2…

<function __main__.do_plot(col)>

# Verify stationarity

In [7]:
# Null hypothesis: the time series is non-stationary
utils.adf_test(sent_scaled.pd_series())

AttributeError: module 'utils' has no attribute 'adf_test'

In [9]:
# Null hypothesis: the time series is stationary
utils.kpss_test(sent_scaled.pd_series())

Results of KPSS Test:


/Users/Xavier/cdpq-ia/utils/utils.py:413: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpsstest = kpss(timeseries, regression="c", nlags="auto")


Test Statistic            0.023708
p-value                   0.100000
Lags Used                25.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64

# Granger Causality

In [10]:
results = statistics.granger_causality_tests(ts_scaled, ts_scaled, maxlag=10)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=-0.0000 , p=1.0000  , df_denom=520, df_num=1
ssr based chi2 test:   chi2=-0.0000 , p=1.0000  , df=1
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=1
parameter F test:         F=55.0121 , p=0.0000  , df_denom=520, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=-0.0000 , p=1.0000  , df_denom=518, df_num=2
ssr based chi2 test:   chi2=-0.0000 , p=1.0000  , df=2
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=2
parameter F test:         F=39.7895 , p=0.0000  , df_denom=518, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=-0.0000 , p=1.0000  , df_denom=516, df_num=3
ssr based chi2 test:   chi2=-0.0000 , p=1.0000  , df=3
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=3
parameter F test:         F=25.7618 , p=0.0000  , df_denom=516, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.0000  , p=1.0000  

/Users/Xavier/cdpq-ia/venv/lib/python3.11/site-packages/darts/utils/statistics.py:462: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  return kpss(ts.values(copy=False), regression, nlags)
/Users/Xavier/cdpq-ia/venv/lib/python3.11/site-packages/darts/utils/statistics.py:462: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  return kpss(ts.values(copy=False), regression, nlags)


In [11]:
utils.load_data_dict()

,FEATURE_NAME,FEATURE_CATEGORY,FEATURE_DESCRIPTION
0,US_CPI,INFLATION,"Consumer Price Index for the United States, me..."
1,US_CPI_EXCL_FOOD_N_ENERGY,INFLATION,Consumer Price Index for the United States exc...
2,US_PERSONAL_SPENDING_PCE,INFLATION,"Personal Consumption Expenditures Price Index,..."
3,CANADA_CPI,INFLATION,"Consumer Price Index for Canada, measuring the..."
4,CANADA_CPI_EXCL_FOOD_N_ENERGY,INFLATION,Consumer Price Index for Canada excluding food...
5,US_CAPACITY_UTILIZATION,GDP_ECONOMIC_GROWTH,Percentage of resources used by corporations a...
6,US_M2,GDP_ECONOMIC_GROWTH,Measure of the money supply that includes cash...
7,US_CONSUMER_CREDIT,GDP_ECONOMIC_GROWTH,Total amount of credit extended to individuals...
8,US_INDUSTRIAL_PRODUCTION,GDP_ECONOMIC_GROWTH,Measure of the output of the industrial sector...
9,US_NFP_ESTABLISHMENT_DATA,GDP_ECONOMIC_GROWTH,"Nonfarm Payrolls data, representing the total ..."
